In [1]:
%matplotlib inline

import dateutil.parser
import datetime
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow. contrib import learn

from pymongo import MongoClient
from bson.objectid import ObjectId

from sklearn.metrics import mean_squared_error

from lstm import lstm_model
from data_processing import generate_data, load_csvdata

In [15]:
LOG_DIR = './ops_logs/lstm_weather'
TIMESTEPS = 10
RNN_LAYERS = [{'num_units': 5}]
DENSE_LAYERS = [10, 10]
TRAINING_STEPS = 100
BATCH_SIZE = 10
PRINT_STEPS = TRAINING_STEPS / 100

In [16]:
def load_stock_frame(filename):
    # load the stock data
    data_raw = pd.read_csv(filename)
    data_raw['FAI'] = data_raw['FAI'].astype(float)
    data_raw['STOCK'] = data_raw['STOCK'].astype(float)
    data_raw['PE'] = data_raw['PE'].astype(float)
    data_raw['TIME'] = data_raw['TIME'].astype(int)
    df = pd.DataFrame(data_raw, columns=['TIME', 'FAI', 'STOCK', 'PE'])
    return df.set_index('TIME')

In [17]:
# scale values to resonable values and convert to float
data_stock = load_stock_frame("mydata/analyisicsv1.csv")
X, y = load_csvdata(data_stock, TIMESTEPS, seperate=False)

In [18]:
regressor = learn.SKCompat(learn.Estimator(
    model_fn=lstm_model(
        TIMESTEPS,
        RNN_LAYERS,
        DENSE_LAYERS
    ),
    model_dir=LOG_DIR
))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb20c5477d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [19]:
# create a lstm instance and validation monitor
validation_monitor = learn.monitors.ValidationMonitor(X['val'], y['val'],
                                                     every_n_steps=PRINT_STEPS,
                                                     early_stopping_rounds=1000)
regressor.fit(X['train'], y['train'],
             monitors=[validation_monitor],
             batch_size=BATCH_SIZE,
             steps=TRAINING_STEPS)

predicted = regressor.predict(X['test'])

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Use `tf.contrib.losses.mean_squared_error` and explicit logits computation.
Instructions for updating:
Use tf.losses.mean_squared_error instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.


TypeError: Existing "global_step" must be a Variable or Tensor: <function get_global_step at 0x7fb2174c4500>.